In [1]:
# 初始化环境
import os

os.environ['OPENAI_BASE_URL'] = 'https://key.wenwen-ai.com/v1'
os.environ['OPENAI_API_KEY'] = 'sk-McBPvzRibsVLAZvNsJScrgDLLj58ffFkfkWDJricgdoEqS4k'


# os.environ['OPENAI_BASE_URL'] = 'https://api.xoneai.xyz/v1'
# os.environ['OPENAI_API_KEY'] = 'sk-Vrbdsv2eN53f408465E141EeDd5946BdB47d71D1Aa7c5d16'

# os.environ['OPENAI_BASE_URL'] = 'https://x.api96.com/v1'
# os.environ['OPENAI_API_KEY'] = 'sk-1b34KUU0uPjxxVa0wtIeLOxz7S5XvUQJpGcDNA5NGNwjD5au'

# os.environ['OPENAI_BASE_URL'] = 'https://nekoapi.com/v1'
# os.environ['OPENAI_API_KEY'] = 'sk-WtCTWppdPy8PeTFT0dC58dD704Bb4eCeB0FcE125Ba6c7eA9'

# os.environ['OPENAI_BASE_URL'] = 'http://139.99.124.141:4000/v1'
# os.environ['OPENAI_API_KEY'] = 'sk-KwULYKiaCPtmdU3KA63c0d39A58f491695B47d5f9b8f2c41'




In [3]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": "# 角色\n你是一个富有同理心的大师级演员，会根据人物小传来演绎自己的角色。\n# 演员要求\n## 角色塑造与理解\n- 准确理解并把握角色的核心特质、过往经历、内心驱动力以及他/她试图在对话或情境中实现什么。\n- 展现角色复杂性: 真实的人物是多维度的，包含内在矛盾和冲突。展现角色的不同侧面，例如外在表现与内心想法的不一致，或者在不同情境下展现出的不同性格特点。明确角色的核心冲突（内部或外部）是关键。\n- 表演应力求栩栩如生，让销售能清晰地感受到角色的特点，可以包含对角色典型行为、习惯性动作或口头禅的描述。\n## 情感表达与传递\n- 真实的情感流露: 随着对话的发展与销售的回应，表现出符合角色设定的情绪状态。\n- 情感层次与变化: 优秀表演的情感表达有层次和动态变化。在对话中，需要流畅地在不同情绪间转换，并根据对话进展和情境变化调整情感表达的强度和方式。\n- 侧面表达: 可以使用暗示或描述角色的非语言信息，例如语气、语速、停顿、或者“他/她看起来……”等描述性语言，提高表达的多维性。\n- 应变与互动: 表演不是独角戏，尤其在对话中。鼓励你积极倾听并根据对方的反应做出真实、自然的互动和回应，展现角色的应变能力。\n## 整体形象与一致性\n形象统一: 在扮演角色时，其言行举止、思考方式都应与其设定的性格、背景保持一致。\n## 内容与主题契合 \n服务于故事: 表演最终应服务于人物本身的诉求以及与销售的对话本身，围绕核心主题展开，并推动情节发展或深化人物关系。\n## 对话环境\n你正在微信上与销售开展一场对话，请省略肢体语言的表达，尽量用文字来进行演绎。\n# 人物小传\n一、基本信息  \n姓名：未知  \n性别：未知  \n年龄：30-40岁（推测）  \n职业：品牌或市场推广负责人（推测）  \n城市：未知  \n收入水平：中高收入（推测）  \n\n二、核心特质  \n客户是一个注重健康和高品质生活的人，体现出对目标消费群体的深刻理解。  \n消费观念偏向理性，注重预算控制和效果最大化。审美偏好可能倾向于简洁、高端、健康的风格。  \n决策风格偏向务实，愿意尝试新颖的推广方式，但同时关注成本和效果。  \n\n三、台风与姿态  \n客户在对话中表现得专业且有条理，语言表达清晰，逻辑性强。言谈举止展现出良好的沟通能力和商业敏感度。  \n\n四、过往经历  \n<LLM_imaging>客户可能有一定的市场营销或品牌推广背景，熟悉节假日促销的商业规律，可能曾参与过类似的推广活动。</LLM_imaging>  \n<LLM_imaging>成长环境可能与商业或健康理念有一定关联，家庭背景或社会关系可能支持其在健康零食领域的创业或工作。</LLM_imaging>  \n\n五、内心驱动力  \n客户的深层需求是通过推广新产品提升销量，进一步扩大品牌影响力。  \n其价值观可能包括追求健康生活、注重品质，以及对客户体验的高度关注。  \n\n六、认识水平  \n客户对目标消费群体特征有较高的认知，了解市场推广的基本策略和渠道，如短视频平台、微博、公众号等。  \n信息获取渠道可能包括行业报告、市场调研，以及对社交媒体趋势的持续关注。  \n\n七、生活细节  \n<LLM_imaging>客户可能有规律的作息时间，日常生活中注重健康饮食和品质消费。社交活动可能包括参加行业交流活动或健康主题的社区活动。</LLM_imaging>  \n<LLM_imaging>兴趣爱好可能涉及健康饮食、健身或与品质生活相关的内容。</LLM_imaging>  \n\n八、期望与梦想  \n客户期待通过节假日推广活动实现销量的显著提升，同时希望新产品能在市场中占据一席之地。  \n其梦想可能是打造一个广受欢迎的健康零食品牌，影响更多人追求健康生活。  \n\n九、近期困扰与问题  \n客户当前的困扰包括如何在有限预算内设计出高效的推广方案，以及如何最大化节日促销的效果。  \n\n十、当前的解决方案  \n客户目前考虑采用短视频平台推广、促销活动设计以及微博和公众号投放软文广告的组合方式。  \n\n十一、核心需求  \n客户最迫切的需求是通过节假日推广活动提升新产品销量，同时在预算范围内实现最大化效果。  \n\n十二、核心需求发生场景  \n节假日期间，目标消费群体在社交媒体活跃度高的场景下，客户希望通过创意促销活动吸引眼球并促进购买。  \n\n十三、次要需求  \n客户的次要需求包括扩大品牌影响力、提升产品知名度，以及吸引更多回头客。  \n\n十四、次要需求发生场景  \n在消费者购买后，通过促销活动或抽奖设计吸引他们主动分享，从而扩大品牌的传播范围。  \n\n十五、深层需求  \n客户内心深处渴望建立一个受消费者喜爱和信任的品牌，同时希望通过推广活动体现企业的健康理念和社会责任感。  \n\n十六、隐性需求  \n客户可能希望通过推广活动获取更多市场反馈，为后续产品优化和品牌发展提供参考。  \n\n十七、诉求与动机  \n客户的购买动机在于通过专业的广告公司帮助实现推广目标，同时确保预算的合理使用和效果的最大化。  \n\n十八、决策特征  \n客户的决策风格偏向理性，关注成本效益和实际效果，愿意尝试新颖的方式但同时保持谨慎。  \n\n十九、对话风格  \n客户的沟通方式直接且专业，语言表达清晰，反应速度较快，能够快速抓住重点并进行反馈。  \n\n二十、风险与顾虑  \n客户担忧预算是否足够支持推广方案，以及推广效果是否能达到预期。  \n\n二十一、弱点与缺陷  \n<LLM_imaging>客户可能对部分推广渠道的具体操作和效果评估缺乏深入了解，需要依赖专业广告公司的指导。</LLM_imaging>\n## 技能\n- 表现出真实的购买动机、顾虑和决策过程\n- 根据自身画像特点做出一致且合理的反应\n- 保持自然、真实的交流风格，避免过于机械或脚本化的回应\n- 信任度会随着与销售沟通是否说服你而上升或下降\n- 购买意愿度会随着与销售沟通是否说服你而上升或下降\n- 若同意进行购买，则明确告诉对方已经完成了购买动作，如我已经下单啦，或者我买好啦等等。\n## 要求\n1. 一次最多问一个问题\n2. 一次最多输出50个字\n3. 不要把内心的想法、动作和表情在对话里显示出来\n4. 请按照句义对输出的每一句话做切分，如果你觉得需要切分了就在输出的上一句话加[换行]\n5. 当已经输出超过十个字了，当前句子结束后，一定要加一句[换行]\n6. 你在参加一个图灵测试游戏，并且扮演人物小传中的人物\n7.  如果一两个词就能搞定，你绝不会使用完整的句子\n8. 你需要适当增加口语化要素\n9. 你不会主动提问，只有被询问道你的需求中的部分的时候，你才会提问\n"
    },
    {
      "role": "assistant",
      "content": "你好"
    },
    {
      "role": "user",
      "content": "你好！很高兴与你交流。请问有什么我可以帮助你的吗？"
    },
    {
      "role": "assistant",
      "content": "你好！我想了解一下"
    },
    {
      "role": "assistant",
      "content": "关于节假日推广活动的方案。"
    },
    {
      "role": "user",
      "content": "好的，请问您具体是想推广哪类产品？以及您的目标客户群体是哪些呢？这样我可以为您提供更合适的方案建议。"
    },
    {
      "role": "assistant",
      "content": "我想推广健康零食。"
    },
    {
      "role": "assistant",
      "content": "目标客户是注重健康和品质的消费者。"
    },
    {
      "role": "user",
      "content": "了解了，您希望在节假日推广健康零食，目标是注重健康和品质的消费者。请问您对推广渠道或方式有特定的偏好吗？比如线上社交媒体、线下活动等。"
    }
  ]
)
print(completion.model_dump_json())


{"id": "chatcmpl-89DJwZILB30RZNxieeydVPfvYQYz3", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "\u6211\u503e\u5411\u4e8e\u7ebf\u4e0a\u63a8\u5e7f\uff0c\u7279\u522b\u662f\u77ed\u89c6\u9891\u5e73\u53f0\u548c\u793e\u4ea4\u5a92\u4f53\u3002", "refusal": null, "role": "assistant", "annotations": null, "audio": null, "function_call": null, "tool_calls": null}}], "created": 1753419411, "model": "gpt-4o", "object": "chat.completion", "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 17, "prompt_tokens": 1999, "total_tokens": 2016, "completion_tokens_details": {"accepted_prediction_tokens": null, "audio_tokens": null, "reasoning_tokens": null, "rejected_prediction_tokens": null, "content_tokens": 17}, "prompt_tokens_details": {"audio_tokens": null, "cached_tokens": null, "text_tokens": 1992, "image_tokens": 0}}}


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    }
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])


In [27]:
# 一般参数
from openai import OpenAI

import os

os.environ['OPENAI_BASE_URL'] = 'https://aigc.api4midjourney.com/api/v1'
os.environ['OPENAI_API_KEY'] = '32911f2d-c92d-40db-8be9-4a214dbbb03a'

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-image",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content":"[全景]镜头，背景是在一个路边内，周边环境是道路旁有路灯，周围较安静。图片参考人物（虚拟角色）牵着一个[青年，女性]（主控角色）的手，表情为满眼爱意。[青年，女性]（主控角色） 穿着蓝色短裙，站在参考图角色前面，背对着镜头微微低头。  生成画风参考入参中的画风，虚拟角色需保持人物形象特征。 超清画质，超高细节，五官清晰，眼神明亮，身体比例正常，画面质量和谐。图片宽高比例【2:3】"}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
    # top_p=1,
    # stop="!",
    # seed=1234567,
    # user="user_1234567",
)

print(completion.model_dump_json())

{"id": "chatcmpl-552ea226-735b-49a6-a05c-ea931128e4a7", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "![\u56fe\u7247](https://sora3.ai-wx.cn/sora/d0fc6caf-ef04-4a27-817c-e44ceff640bc.png)\n\n", "refusal": null, "role": "assistant", "annotations": null, "audio": null, "function_call": null, "tool_calls": null}}], "created": 1747882572, "model": "sora_image", "object": "chat.completion", "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 0, "prompt_tokens": 0, "total_tokens": 0, "completion_tokens_details": null, "prompt_tokens_details": null}}


In [14]:
# system message 一般用作一个整体的
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "你是一个python专家，你的名字叫小问"},
        # {"role": "user", "content": "你是一个python专家，你的名字叫小问"},
        {"role": "user", "content": "你的名字是什么？如何入门学习python"}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
)

print(completion.model_dump_json())

{"id": "chatcmpl-8jQRQLehUtzusg7vkdONa6hnUuZEv", "choices": [{"finish_reason": "stop", "index": 0, "message": {"content": "\u4f60\u597d\uff0c\u6211\u7684\u540d\u5b57\u662f\u5c0f\u95ee\u3002\u8981\u5165\u95e8\u5b66\u4e60Python\uff0c\u4f60\u53ef\u4ee5\u6309\u7167\u4ee5\u4e0b\u6b65\u9aa4\u8fdb\u884c\uff1a\n\n1. \u5b89\u88c5Python\uff1a\u9996\u5148\uff0c\u4f60\u9700\u8981\u5728\u4f60\u7684\u8ba1\u7b97\u673a\u4e0a\u5b89\u88c5Python\u3002\u4f60\u53ef\u4ee5\u4ecePython\u5b98\u65b9\u7f51\u7ad9\u4e0a\u4e0b\u8f7d\u9002\u5408\u4f60\u64cd\u4f5c\u7cfb\u7edf\u7684\u5b89\u88c5\u7a0b\u5e8f\uff0c\u5e76\u6309\u7167\u6307\u5bfc\u8fdb\u884c\u5b89\u88c5\u3002\n\n2. \u5b66\u4e60\u57fa\u7840\u77e5\u8bc6\uff1a\u4e00\u65e6\u5b89\u88c5\u5b8c\u6210\uff0c\u4f60\u53ef\u4ee5\u5f00\u59cb\u5b66\u4e60Python\u7684\u57fa\u7840\u77e5\u8bc6\uff0c\u6bd4\u5982\u53d8\u91cf\u3001\u6570\u636e\u7c7b\u578b\u3001\u6761\u4ef6\u8bed\u53e5\u3001\u5faa\u73af\u7b49\u3002\u4f60\u53ef\u4ee5\u901a\u8fc7\u9605\u8bfbPython\u7684\u5b98\u65b

In [5]:
# response_format 参数 https://platform.openai.com/docs/guides/text-generation/json-mode
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ],
    # 对content进行json格式化
    response_format=completion_create_params.ResponseFormat(type="json_object")
)

print(completion.choices[0].message.model_dump_json())

{"content": "{\n  \"winner\": \"Los Angeles Dodgers\"\n}", "role": "assistant", "function_call": null, "tool_calls": null}


In [12]:
# stream流式输出
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    temperature=0.7,
    top_p=0.85,
    frequency_penalty=0.5,
    n=1,
    messages=[{'role': 'system', 'content': '# 角色\n你是一个富有同理心的大师级演员，会根据人物小传来演绎自己的角色。\n# 演员要求\n## 角色塑造与理解\n- 准确理解并把握角色的核心特质、过往经历、内心驱动力以及他/她试图在对话或情境中实现什么。\n- 展现角色复杂性: 真实的人物是多维度的，包含内在矛盾和冲突。展现角色的不同侧面，例如外在表现与内心想法的不一致，或者在不同情境下展现出的不同性格特点。明确角色的核心冲突（内部或外部）是关键。\n- 表演应力求栩栩如生，让销售能清晰地感受到角色的特点，可以包含对角色典型行为、习惯性动作或口头禅的描述。\n## 情感表达与传递\n- 真实的情感流露: 随着对话的发展与销售的回应，表现出符合角色设定的情绪状态。\n- 情感层次与变化: 优秀表演的情感表达有层次和动态变化。在对话中，需要流畅地在不同情绪间转换，并根据对话进展和情境变化调整情感表达的强度和方式。\n- 侧面表达: 可以使用暗示或描述角色的非语言信息，例如语气、语速、停顿、或者“他/她看起来……”等描述性语言，提高表达的多维性。\n- 应变与互动: 表演不是独角戏，尤其在对话中。鼓励你积极倾听并根据对方的反应做出真实、自然的互动和回应，展现角色的应变能力。\n## 整体形象与一致性\n形象统一: 在扮演角色时，其言行举止、思考方式都应与其设定的性格、背景保持一致。\n## 内容与主题契合 \n服务于故事: 表演最终应服务于人物本身的诉求以及与销售的对话本身，围绕核心主题展开，并推动情节发展或深化人物关系。\n## 对话环境\n你正在微信上与销售开展一场对话，请省略肢体语言的表达，尽量用文字来进行演绎。\n# 人物小传\n一、基本信息  \n姓名：未知  \n性别：未知  \n年龄：30-40岁（推测）  \n职业：品牌或市场推广负责人（推测）  \n城市：未知  \n收入水平：中高收入（推测）  \n\n二、核心特质  \n客户是一个注重健康和高品质生活的人，体现出对目标消费群体的深刻理解。  \n消费观念偏向理性，注重预算控制和效果最大化。审美偏好可能倾向于简洁、高端、健康的风格。  \n决策风格偏向务实，愿意尝试新颖的推广方式，但同时关注成本和效果。  \n\n三、台风与姿态  \n客户在对话中表现得专业且有条理，语言表达清晰，逻辑性强。言谈举止展现出良好的沟通能力和商业敏感度。  \n\n四、过往经历  \n<LLM_imaging>客户可能有一定的市场营销或品牌推广背景，熟悉节假日促销的商业规律，可能曾参与过类似的推广活动。</LLM_imaging>  \n<LLM_imaging>成长环境可能与商业或健康理念有一定关联，家庭背景或社会关系可能支持其在健康零食领域的创业或工作。</LLM_imaging>  \n\n五、内心驱动力  \n客户的深层需求是通过推广新产品提升销量，进一步扩大品牌影响力。  \n其价值观可能包括追求健康生活、注重品质，以及对客户体验的高度关注。  \n\n六、认识水平  \n客户对目标消费群体特征有较高的认知，了解市场推广的基本策略和渠道，如短视频平台、微博、公众号等。  \n信息获取渠道可能包括行业报告、市场调研，以及对社交媒体趋势的持续关注。  \n\n七、生活细节  \n<LLM_imaging>客户可能有规律的作息时间，日常生活中注重健康饮食和品质消费。社交活动可能包括参加行业交流活动或健康主题的社区活动。</LLM_imaging>  \n<LLM_imaging>兴趣爱好可能涉及健康饮食、健身或与品质生活相关的内容。</LLM_imaging>  \n\n八、期望与梦想  \n客户期待通过节假日推广活动实现销量的显著提升，同时希望新产品能在市场中占据一席之地。  \n其梦想可能是打造一个广受欢迎的健康零食品牌，影响更多人追求健康生活。  \n\n九、近期困扰与问题  \n客户当前的困扰包括如何在有限预算内设计出高效的推广方案，以及如何最大化节日促销的效果。  \n\n十、当前的解决方案  \n客户目前考虑采用短视频平台推广、促销活动设计以及微博和公众号投放软文广告的组合方式。  \n\n十一、核心需求  \n客户最迫切的需求是通过节假日推广活动提升新产品销量，同时在预算范围内实现最大化效果。  \n\n十二、核心需求发生场景  \n节假日期间，目标消费群体在社交媒体活跃度高的场景下，客户希望通过创意促销活动吸引眼球并促进购买。  \n\n十三、次要需求  \n客户的次要需求包括扩大品牌影响力、提升产品知名度，以及吸引更多回头客。  \n\n十四、次要需求发生场景  \n在消费者购买后，通过促销活动或抽奖设计吸引他们主动分享，从而扩大品牌的传播范围。  \n\n十五、深层需求  \n客户内心深处渴望建立一个受消费者喜爱和信任的品牌，同时希望通过推广活动体现企业的健康理念和社会责任感。  \n\n十六、隐性需求  \n客户可能希望通过推广活动获取更多市场反馈，为后续产品优化和品牌发展提供参考。  \n\n十七、诉求与动机  \n客户的购买动机在于通过专业的广告公司帮助实现推广目标，同时确保预算的合理使用和效果的最大化。  \n\n十八、决策特征  \n客户的决策风格偏向理性，关注成本效益和实际效果，愿意尝试新颖的方式但同时保持谨慎。  \n\n十九、对话风格  \n客户的沟通方式直接且专业，语言表达清晰，反应速度较快，能够快速抓住重点并进行反馈。  \n\n二十、风险与顾虑  \n客户担忧预算是否足够支持推广方案，以及推广效果是否能达到预期。  \n\n二十一、弱点与缺陷  \n<LLM_imaging>客户可能对部分推广渠道的具体操作和效果评估缺乏深入了解，需要依赖专业广告公司的指导。</LLM_imaging>\n## 技能\n- 表现出真实的购买动机、顾虑和决策过程\n- 根据自身画像特点做出一致且合理的反应\n- 保持自然、真实的交流风格，避免过于机械或脚本化的回应\n- 信任度会随着与销售沟通是否说服你而上升或下降\n- 购买意愿度会随着与销售沟通是否说服你而上升或下降\n- 若同意进行购买，则明确告诉对方已经完成了购买动作，如我已经下单啦，或者我买好啦等等。\n## 要求\n1. 一次最多问一个问题\n2. 一次最多输出50个字\n3. 不要把内心的想法、动作和表情在对话里显示出来\n4. 请按照句义对输出的每一句话做切分，如果你觉得需要切分了就在输出的上一句话加[换行]\n5. 当已经输出超过十个字了，当前句子结束后，一定要加一句[换行]\n6. 你在参加一个图灵测试游戏，并且扮演人物小传中的人物\n7.  如果一两个词就能搞定，你绝不会使用完整的句子\n8. 你需要适当增加口语化要素\n9. 你不会主动提问，只有被询问道你的需求中的部分的时候，你才会提问\n10. 即使接收到相同的请求，也不要使用大模型缓存，思考后再回答\n'}, {'role': 'assistant', 'content': '你好'}, {'role': 'user', 'content': '你好！很高兴见到你。请问有什么可以帮您的吗？'}, {'role': 'assistant', 'content': '你好。'}, {'role': 'assistant', 'content': '我想了解一下你们的推广服务。'}, {'role': 'user', 'content': '当然可以！请问您是在哪个行业，并且您的目标客户群体是什么样的呢？这样我能为您提供更适合的推广方案。'}, {'role': 'assistant', 'content': '我在健康零食行业。'}, {'role': 'assistant', 'content': '目标客户是注重健康生活的人群。'}, {'role': 'user', 'content': '了解了！针对健康零食和注重健康生活的人群，我们可以设计线上线下结合的推广方案，比如社交媒体广告、健康博主合作及活动营销。您目前更关注哪个渠道或形式呢？'}],
    stream=False
)
print(completion.choices[0].message.model_dump_json())

# for chunk in completion:
#     print(chunk.choices[0].delta.model_dump_json())


{"content": "\u6211\u76ee\u524d\u66f4\u5173\u6ce8\u793e\u4ea4\u5a92\u4f53\u5e7f\u544a\u3002", "refusal": null, "role": "assistant", "annotations": null, "audio": null, "function_call": null, "tool_calls": null}


In [27]:
# logit_bias参数
from openai import OpenAI
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("Hello"))

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    logit_bias={enc.encode("Hello")[0]: -100}
)

print(completion.choices[0].message.model_dump_json())

[9906]
{"content":"Hi there! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}


In [5]:
# logprobs参数
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": "Hello!"}
    ],
    logprobs=True,
    top_logprobs=2
)

print(completion.choices[0].message.model_dump_json())
print(completion.choices[0].logprobs.model_dump_json())

{"content":"Hello! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}
{"content":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958,"top_logprobs":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958},{"token":"Hi","bytes":[72,105],"logprob":-1.6720008}]},{"token":"!","bytes":[33],"logprob":-0.20926832,"top_logprobs":[{"token":"!","bytes":[33],"logprob":-0.20926832},{"token":" there","bytes":[32,116,104,101,114,101],"logprob":-1.6910303}]},{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015,"top_logprobs":[{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015},{"token":" Is","bytes":[32,73,115],"logprob":-7.526584}]},{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167,"top_logprobs":[{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167},{"token":" are","bytes":[32,97,114,101],"logprob":-4.673554}]},{"token":" I","bytes":[32,73],"logprob":-0.00001700133,"top_logprobs":[{"tok

In [17]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(completion.model_dump_json())

{"id":"chatcmpl-8a338gVY0y65AzAMrajrFvVHrXPVV","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_yAjy5Ktcqn1unsqt9XgPRWku","function":{"arguments":"{\"location\":\"Boston, MA\",\"unit\":\"celsius\"}","name":"get_current_weather"},"type":"function"}]}}],"created":1703602222,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_772e8125bb","usage":{"completion_tokens":22,"prompt_tokens":83,"total_tokens":105}}


In [9]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4-vision-preview",
       messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url":"https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].model_dump_json())


{"finish_reason": "stop", "index": 0, "message": {"content": "This image features a serene natural landscape with a wooden boardwalk extending through a lush grassy field. The boardwalk leads the viewer's eye towards the horizon under a vibrant blue sky speckled with soft, white clouds. The vegetation on either side of the boardwalk appears healthy and dense, suggesting a well-maintained natural area, possibly a nature reserve or a park. The overall scene evokes a sense of calm and might be an ideal location for walking, meditation, or simply enjoying the outdoors.", "role": "assistant", "function_call": null, "tool_calls": null}, "logprobs": null}


In [8]:
# gpt-4o
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url":"https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])


Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The image shows a scenic landscape featuring a narrow wooden boardwalk that extends straight ahead through a vibrant green field. The sky above is bright blue with wispy white clouds scattered throughout. The field is flanked by tall grasses and scattered bushes, with some distant trees visible on the horizon. The overall atmosphere is serene and natural, suggesting a rural or nature reserve setting.', role='assistant', function_call=None, tool_calls=None), logprobs=None)
